In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-01-22 02:27:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-01-22 02:27:55 (5.99 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url="https://sj-db-data.s3.us-east-2.amazonaws.com/hotel_review_data_sql.csv"
spark.sparkContext.addFile(url)
user_data_df=spark.read.csv(SparkFiles.get("hotel_review_data_sql.csv"),sep=',',header=True,inferSchema=True)
user_data_df.show()

+---+--------------------+--------------------+-----------------+-------+------------------+-------------------+--------------------+----------+--------+--------------+--------------------+
| id|            hotel_id|             address|             city|country|          latitude|          longitude|                name|postalCode|province|reviews_rating|  reviews_sourceURLs|
+---+--------------------+--------------------+-----------------+-------+------------------+-------------------+--------------------+----------+--------+--------------+--------------------+
|  1|AWE2FvX5RxPSIh2RscTK|     5620 Calle Real|           Goleta|     US|          34.44178|         -119.81979|Best Western Plus...|     93117|      CA|             3|https://www.tripa...|
|  2|AVwcj_OhkufWRAb5wi9T|5th And San Carlo...|Carmel by the Sea|     US|          36.55722|-121.92193999999999|Best Western Carm...|     93921|      CA|             4|https://www.tripa...|
|  3|AVwcj_OhkufWRAb5wi9T|5th And San Carlo...|Car

In [5]:
hotel_reviews_info= user_data_df.select(["id", "hotel_id", "address", "city", "country","latitude","longitude","name","postalcode","province","reviews_rating","reviews_sourceURLs"])
hotel_reviews_info.show(5)

+---+--------------------+--------------------+-----------------+-------+--------+-------------------+--------------------+----------+--------+--------------+--------------------+
| id|            hotel_id|             address|             city|country|latitude|          longitude|                name|postalcode|province|reviews_rating|  reviews_sourceURLs|
+---+--------------------+--------------------+-----------------+-------+--------+-------------------+--------------------+----------+--------+--------------+--------------------+
|  1|AWE2FvX5RxPSIh2RscTK|     5620 Calle Real|           Goleta|     US|34.44178|         -119.81979|Best Western Plus...|     93117|      CA|             3|https://www.tripa...|
|  2|AVwcj_OhkufWRAb5wi9T|5th And San Carlo...|Carmel by the Sea|     US|36.55722|-121.92193999999999|Best Western Carm...|     93921|      CA|             4|https://www.tripa...|
|  3|AVwcj_OhkufWRAb5wi9T|5th And San Carlo...|Carmel by the Sea|     US|36.55722|-121.9219399999999

In [10]:
# Configuration for RDS instance
mode="overwrite"
jdbc_url = "jdbc:postgresql://sj-db-1.ctg25floyats.us-east-2.rds.amazonaws.com:5432/Hotel_Reviews_db"
config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}



In [11]:
# Write DataFrame to table

hotel_reviews_info.write.jdbc(url=jdbc_url, table='review', mode=mode, properties=config)